### - 자연어 처리 테스트

- 자연어 처리..

#### 1. 데이터 읽기

In [ ]:
import pandas as pd

jeju_poi = pd.read_excel('./data/220119/_제주도_POI_컬럼 정리.xlsx', index_col=0)

In [ ]:
jeju_poi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2306 entries, 0 to 5740
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              2306 non-null   object 
 1   address_name         2306 non-null   object 
 2   category_group_name  2306 non-null   object 
 3   category_name        2306 non-null   object 
 4   id                   2306 non-null   int64  
 5   place_name           2306 non-null   object 
 6   x                    2306 non-null   float64
 7   y                    2306 non-null   float64
 8   rating               2306 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 180.2+ KB


In [ ]:
jeju_poi.head(2)

,keyword,address_name,category_group_name,category_name,id,place_name,x,y,rating
0,테마파크,제주특별자치도 제주시 연동 1320,관광명소,"문화,예술 > 문화시설 > 박물관",26388484,수목원테마파크 아이스뮤지엄,126.488398,33.470777,1.0
1,"테마파크,가볼만한 곳",제주특별자치도 제주시 애월읍 신엄리 2880-12,관광명소,"여행 > 관광,명소 > 테마파크",1129394481,고스트타운,126.356936,33.476195,3.4


#### 2. 자연어 선택

- keyword와 category_group_name 사용

In [ ]:
keywords = set()
for keyword in jeju_poi['keyword'].unique():
    for k in keyword.split(','):
        keywords.add(k)
keywords = list(keywords)

In [ ]:
keywords

['레포츠', '가볼만한 곳', '맛집', '감성', '오션뷰', '테마파크', '가족여행', '체험', '분위기 좋은']

In [ ]:
groups = []
for category in jeju_poi['category_group_name'].unique():
    groups.append(category)

In [ ]:
groups

['관광명소', '음식점', '카페']

In [ ]:
jeju_poi[['id', 'place_name', 'keyword', 'category_group_name']].head(2)

,id,place_name,keyword,category_group_name
0,26388484,수목원테마파크 아이스뮤지엄,테마파크,관광명소
1,1129394481,고스트타운,"테마파크,가볼만한 곳",관광명소


#### 3. content 만들기

- keyword와 category_group_name를 합쳐 content 컬럼을 추가한다.
  - 자연어를 이용한 유사도 측정을 위해

In [ ]:
from tqdm.notebook import tqdm
jeju_poi['content'] = ''

for idx, row in tqdm(jeju_poi.iterrows()):
    content = (row['keyword']+','+row['category_group_name']).replace(' ', '').replace(',', ' ')
    jeju_poi.loc[idx, 'content'] = content

0it [00:00, ?it/s]

In [ ]:
jeju_poi[['id', 'place_name', 'keyword', 'category_group_name', 'content']].head(2)

,id,place_name,keyword,category_group_name,content
0,26388484,수목원테마파크 아이스뮤지엄,테마파크,관광명소,테마파크 관광명소
1,1129394481,고스트타운,"테마파크,가볼만한 곳",관광명소,테마파크 가볼만한곳 관광명소


In [ ]:
jeju_poi.loc[0, 'content']

'테마파크 관광명소'

#### 4. CountVectorizer

- 새로 생선한 'content'를 기준으로 CountVectorizer 진행

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
content_mat = count_vect.fit_transform(jeju_poi['content'])
content_mat.shape

(2306, 38)

In [ ]:
count_vect.get_feature_names_out()

array(['가볼만한곳', '가볼만한곳 관광명소', '가족여행', '가족여행 관광명소', '가족여행 음식점', '감성',
       '감성 가볼만한곳', '감성 맛집', '감성 음식점', '감성 카페', '관광명소', '레포츠', '레포츠 가볼만한곳',
       '맛집', '맛집 감성', '맛집 분위기좋은', '맛집 오션뷰', '맛집 음식점', '맛집 카페', '분위기좋은',
       '분위기좋은 감성', '분위기좋은 맛집', '분위기좋은 오션뷰', '분위기좋은 음식점', '분위기좋은 카페',
       '오션뷰', '오션뷰 맛집', '오션뷰 음식점', '오션뷰 카페', '음식점', '체험', '체험 가볼만한곳',
       '카페', '테마파크', '테마파크 가볼만한곳', '테마파크 관광명소', '테마파크 레포츠', '테마파크 음식점'],
      dtype=object)

#### 5. Cosine Similarity

- content_mat을 활용하여 코사인 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

content_sim = cosine_similarity(content_mat, content_mat)
print(content_sim.shape)
print(content_sim[:2])

(2306, 2306)
[[1.         0.51639778 0.51639778 ... 0.         0.         0.        ]
 [0.51639778 1.         1.         ... 0.         0.         0.        ]]


In [ ]:
content_sim_sorted_idx = content_sim.argsort()[:, ::-1]
print(content_sim_sorted_idx[:2])

[[   0 1083   82 ... 1526 1527 1152]
 [  81  610  887 ... 1526 1527 1152]]


In [ ]:
for idx, key in enumerate(content_sim_sorted_idx[0]):
    if content_sim[0][key] < content_sim[0][0]:
        print(content_sim[0][key], idx)
        break
content_sim[:2]

0.5163977794943223 11


array([[1.        , 0.51639778, 0.51639778, ..., 0.        , 0.        ,
        0.        ],
       [0.51639778, 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
import numpy as np

# np.save('content_sim_test.npy', content_sim)

##### ◽ㅇ

### - 서귀포 데이터 : 잘못된 이미지 삭제

In [ ]:
import pandas as pd

seogipo_poi = pd.read_excel('./data/220120/서귀포_POI_이미지 정리(전).xlsx', index_col=0)

In [ ]:
seogipo_poi.head(1)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,rating,img_link
0,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 법환동 877-3,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,10472331,064-739-8254,세리월드,http://place.map.kakao.com/10472331,제주특별자치도 서귀포시 법환상로2번길 97-17,126.511874,33.247016,3.7,https://img1.kakaocdn.net/relay/local/R640x320...


#### 1. 사용할 이미지 확인

- 로고, 장소와 다른 이미지를 삭제하고 남은 이미지 확인
  - 파일명 : (idx)_(plac ename).jpg
- .jpg를 삭제하고 '_'를 기준으로 split

In [ ]:
import os

path = './data/220120/_이미지_서귀포/'
use_idx = []
for dir_name in os.listdir(path):
    if '삭제' in dir_name:
        continue
    print(dir_name)
    for image in os.listdir(path + dir_name):
        image = image.replace('.jpg', '')
        idx, p_name = image.split('_')
        use_idx.append(int(idx))

관광명소
음식점
카페


In [ ]:
use_idx[:10], len(use_idx)

([0, 1039, 1077, 1080, 1083, 1084, 1086, 1087, 1089, 1091], 702)

#### 2. 남은 이미지만 사용

- use_idx를 이용하여 해당하는 인덱스만 사용

In [ ]:
use_idx.sort()

In [ ]:
seogipo_poi = seogipo_poi.loc[use_idx, :].copy()

In [ ]:
seogipo_poi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 702 entries, 0 to 1576
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              702 non-null    object 
 1   address_name         702 non-null    object 
 2   category_group_code  702 non-null    object 
 3   category_group_name  702 non-null    object 
 4   category_name        702 non-null    object 
 5   distance             0 non-null      float64
 6   id                   702 non-null    int64  
 7   phone                514 non-null    object 
 8   place_name           702 non-null    object 
 9   place_url            702 non-null    object 
 10  road_address_name    621 non-null    object 
 11  x                    702 non-null    float64
 12  y                    702 non-null    float64
 13  rating               702 non-null    float64
 14  img_link             702 non-null    object 
dtypes: float64(4), int64(1), object(10)
mem

In [ ]:
seogipo_poi.head(1)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,rating,img_link
0,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 법환동 877-3,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,10472331,064-739-8254,세리월드,http://place.map.kakao.com/10472331,제주특별자치도 서귀포시 법환상로2번길 97-17,126.511874,33.247016,3.7,https://img1.kakaocdn.net/relay/local/R640x320...


#### 3. 데이터 저장

In [ ]:
# seogipo_poi.to_excel('./data/220120/_서귀포_POI_final_use.xlsx')

### - 서귀포 데이터 : 이미지 RESIZE

- 이미지의 크기가 다르기 때문에 동일한 크기로 조절
  - (224, 224)

#### 1. 이미지 Path 확인

- 폴더별로 이미지가 나누어져 있으므로 폴더를 key 값으로 dict형 구성

In [ ]:
import os

path = './data/220120/_서귀포_이미지/'
image_path = {'관광명소' : [], '음식점' : [], '카페' : []}
for dir_name in os.listdir(path):
    print(dir_name)
    for image in os.listdir(path + dir_name):
        image_path[dir_name].append(path + dir_name + '/' + image)

관광명소
음식점
카페


In [ ]:
image_path['관광명소'][:2], image_path['음식점'][:2], image_path['카페'][:2]

(['./data/220120/_서귀포_이미지/관광명소/0_세리월드.jpg',
  './data/220120/_서귀포_이미지/관광명소/1039_신례천생태로2코스.jpg'],
 ['./data/220120/_서귀포_이미지/음식점/1001_서귀포밀면.jpg',
  './data/220120/_서귀포_이미지/음식점/1003_토평 해뜨는집.jpg'],
 ['./data/220120/_서귀포_이미지/카페/1028_사우스포레스트.jpg',
  './data/220120/_서귀포_이미지/카페/1073_네이처캔버스.jpg'])

#### 2. 이미지 Resize

##### ◽PIL 사용

- jpg의 경우 중간에 오류가 발생하여 png로 변경

In [ ]:
from PIL import Image

home = 'C:/Users/Isanghada/Desktop/서귀포_테스트/'
for kind in image_path.keys():
    for image in image_path[kind]:    
        name = image.split('/')[-1]
        img = Image.open(image)
        img.resize((224, 224)).save(home+kind+'/'+name[:-3]+'png', 'png')

#### 3. 이미지명 엑셀 파일

In [ ]:
import os

home = 'C:/Users/Isanghada/Desktop/서귀포_테스트/'
dir_list = []
img_list = []
for dir_name in os.listdir(home):
    print(dir_name)
    for img in os.listdir(home+dir_name):
        dir_list.append(dir_name)
        img_list.append(img)

관광명소
음식점
카페


In [ ]:
len(dir_list), len(img_list)

(702, 702)

In [ ]:
import pandas as pd

all_image = pd.DataFrame({
    'dir' : dir_list,
    'image' : img_list
})

In [ ]:
# all_image.to_excel('./data/220120/서귀포_image_info.xlsx')

##### ◽ㅇ